In [ ]:
"""
Objective:
My objective is to use Python to construct an approximate nearest neighbors algorithm from scratch.
I am using a datset of movie ratings (~100,000 ratings) found here: https://grouplens.org/datasets/movielens/.
In addition to building these algorithms, I will experiment with making changes to the standard models in order to
gain insight and facts into how it is working and/or improve the performance of the model via nonstandard techniques.

"""

In [193]:
"""
load ratings data
"""
import os
import csv
import numpy as np
import pandas as pd

os.chdir("C:/Users/ramy_/Documents/Graduate School/Semester 1/Personalization Theory and Application/ml-latest-small/")


data = []

with open('ratings.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for i, row in enumerate(spamreader):
        data.append(row)

print (data[0:20])
print (data[1][1])

[['userId', 'movieId', 'rating', 'timestamp'], ['1', '31', '2.5', '1260759144'], ['1', '1029', '3', '1260759179'], ['1', '1061', '3', '1260759182'], ['1', '1129', '2', '1260759185'], ['1', '1172', '4', '1260759205'], ['1', '1263', '2', '1260759151'], ['1', '1287', '2', '1260759187'], ['1', '1293', '2', '1260759148'], ['1', '1339', '3.5', '1260759125'], ['1', '1343', '2', '1260759131'], ['1', '1371', '2.5', '1260759135'], ['1', '1405', '1', '1260759203'], ['1', '1953', '4', '1260759191'], ['1', '2105', '4', '1260759139'], ['1', '2150', '3', '1260759194'], ['1', '2193', '2', '1260759198'], ['1', '2294', '2', '1260759108'], ['1', '2455', '2.5', '1260759113'], ['1', '2968', '1', '1260759200']]
31


In [194]:
"""
put data into proper format: pivot table by doing grouby on userID columns
"""
data = np.asarray(data)
data = pd.DataFrame(data=data[1:,:], columns=data[0,0:])

data = data.drop('timestamp', 1)

#print (data.head)
print (data.columns.values)

data = data.pivot(index='userId', columns='movieId')['rating']

print (data.head)
print (data.shape)

['userId' 'movieId' 'rating']
<bound method DataFrame.head of movieId     1    10   100 100017 100032 100034 100083 100106 100159 100163  \
userId                                                                       
1        None  None  None   None   None   None   None   None   None   None   
10       None  None  None   None   None   None   None   None   None   None   
100         4  None  None   None   None   None   None   None   None   None   
101      None  None  None   None   None   None   None   None   None   None   
102      None  None  None   None   None   None   None   None   None   None   
103      None  None  None   None   None   None   None   None   None   None   
104      None  None  None   None   None   None   None   None   None   None   
105      None   3.5  None   None   None   None   None   None   None   None   
106         4  None  None   None   None   None   None   None   None   None   
107      None  None  None   None   None   None   None   None   None   None   
10

In [72]:
"""
calculate sparsity
"""
total = 0
rated = 0
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        total += 1
        if data.iloc[i, j] != None:
            rated += 1

print ((float(rated)/total)*100)
        

1.64391416087


In [195]:
"""
create training and evaluation sets
"""

import random

evaluation_set_size = 100

initial_indices = []
for i in range(data.shape[0]):
    initial_indices.append(i)

indices = []
while len(indices) < evaluation_set_size:
    number = random.choice(initial_indices)
    if number not in indices:
        indices.append(number)

data = np.asarray(data)
print (data.shape)
eval = []
for i in range(data.shape[0]):
    if i in indices:
        eval.append(data[i][:])
eval = np.asarray(eval)
print (eval.shape)
print (eval)
data = np.delete(data, indices, axis=0)
print (data.shape)

(671L, 9066L)
(100L, 9066L)
[[None None None ..., None None None]
 [None '4' None ..., None None None]
 [None None None ..., None None None]
 ..., 
 [None '3' None ..., None None None]
 [None None None ..., None None None]
 [None None None ..., None None None]]
(571L, 9066L)


In [196]:
"""
strip off ratings from the evaluation set
"""
import copy

ratio_of_ratings_missing = 0.5

full_ratings_eval = copy.deepcopy(eval)
for j, row in enumerate(eval):
    indices = []
    for i, rating in enumerate(row):
        if rating != None:
            indices.append(i)
    total_ratings = len(indices)
    while len(indices) > total_ratings*ratio_of_ratings_missing:
        number = random.choice(indices)
        indices.remove(number)
    for i, rating in enumerate(row):
        if i in indices:
            eval[j][i] = None
            
        



In [197]:
"""
identify k nearest neighbors of every user in the evaluation set
"""

k = 5

dictionaries = [None] * evaluation_set_size
for i, eval_row in enumerate(eval):
    dictionaries[i] = {}
    for ref_row in data:
        measure = 0
        count = 0
        for j in range(len(eval_row)):
            if eval_row[j] != None and ref_row[j] != None:
                count += 1
                measure += (float(eval_row[j]) - float(ref_row[j]))**2
        if len(dictionaries[i]) < 5 and count>0:
            #print (measure)
            dictionaries[i][tuple(ref_row.tolist())] = measure
        else:
            if count>0 and measure < max(dictionaries[i].values()):
                #print ("measure")
                #print (dictionaries[i].values())
                #print (measure)
                #print (max(dictionaries[i].values()))
                for key in list(dictionaries[i].keys()):
                    if dictionaries[i][key] == max(dictionaries[i].values()):
                        del dictionaries[i][key]
                        break
                dictionaries[i][tuple(ref_row.tolist())] = measure
                #print (dictionaries[i].values())
    

In [198]:
"""
make predictions on missing ratings based on nearest neighbors
"""

record = copy.deepcopy(eval)

for i, eval_row in enumerate(eval):
    for j in range(len(eval_row)):
        if eval_row[j] == None and full_ratings_eval[i][j] != None:
            total = 0
            the_sum = 0
            for v in dictionaries[i].keys():
                if v[j] != None:
                    the_sum += float(v[j])
                    total += 1
            if total == 0:
                eval[i][j] = 3
            else:
                eval[i][j] = float(the_sum)/float(total)
            
                

In [203]:
"""
calculate standard error among predicted ratings
"""

count = 0
error = 0
for i in range(len(eval)):
    for j in range(len(eval[i])):
        if eval[i][j] != None and record[i][j] == None:
            count += 1
            error += abs(float(full_ratings_eval[i][j]) - float(eval[i][j]))
            #print ("original")
            #print (full_ratings_eval[i][j])
            #print (eval[i][j])
print (count)
print (float(error)/float(count))

9498
0.960047729347


In [219]:
"""
calculate popularity of movies in dataset
"""
counts = [0] * data.shape[1]
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i][j] != None:
            counts[j] += 1

print (max(counts))
print (min(counts))
print (len(counts))
print (sum(counts)/len(counts))
print (np.percentile(counts, 95))

287
0
9066
8
41.0


In [226]:
"""
calculate standard error among predicted ratings only among a subset of predictions
based on whether the movie whose rating is being predicted is popular or not
"""
count = 0
error = 0
for i in range(len(eval)):
    for j in range(len(eval[i])):
        if eval[i][j] != None and record[i][j] == None:
            if counts[j] <= np.percentile(counts, 90):
                count += 1
                error += abs(float(full_ratings_eval[i][j]) - float(eval[i][j]))
            #print ("original")
            #print (full_ratings_eval[i][j])
            #print (eval[i][j])
print (count)
print (float(error)/float(count))

4502
0.902765437583


In [233]:
"""
calculate how prolific users are in evaluation set
"""
users = [0] * record.shape[0]
for i in range(record.shape[0]):
    for j in range(record.shape[1]):
        if record[i][j] != None:
            users[i] += 1

print (max(users))
print (min(users))
print (len(users))
print (sum(users)/len(users))
print (np.percentile(users, 50))

1196
10
100
95
44.5


In [238]:
"""
calculate standard error among predicted ratings only among a subset of predictions
based on how prolific the user who made the predictions is
"""
count = 0
error = 0
for i in range(len(eval)):
    for j in range(len(eval[i])):
        if eval[i][j] != None and record[i][j] == None:
            if users[i] > np.percentile(users, 90):
                count += 1
                error += abs(float(full_ratings_eval[i][j]) - float(eval[i][j]))
            #print ("original")
            #print (full_ratings_eval[i][j])
            #print (eval[i][j])
print (count)
print (float(error)/float(count))

4783
0.91642274723


In [ ]:
"""
Results, insights and discussion
---------------------------------

Benchmark:
data: 100k ratings, 9k movies, 671 users
sparsity (percentage of ratings of movies present in datasest): 1.64%
train/test split: 571 users for training set, 100 users for evaluation set
level of knowledge on evaluation set: 50% of user's ratings missing
value of k for knn: 5
default guess (if none of user's neighbors have rated a movie): 3 out of 5

Benchmark results:
total number of predictions made: 9498
standard error from true rating: 0.96 out of 5

Insights:
among top 10% popular movies (by frequency of ratings):
total number of predictions made: 4996
standard error from true rating: 1.01 out of 5

among bottom 90% popular movies (by frequency of ratings):
total number of predictions made: 4502
standard error from true rating: 0.902 out of 5


The above observation is that if a movie is more popular, the algorithm will have a slightly higher error making
a prediction on it, while if a movie is less popular, the algorithm will have a lower error making a prediction on it.
This is due to the fact that in a sense there are more widespread opinions about popular movies. If a user agrees
with another user on particular type of unpopular movie, that does not necessarily tell us that they will share
a similar opinion on the movie that everyone is familiar with.
It is also the case that with unpopular movies, the error rates were better. This is due to the fact that unpopular
movies can more easily fit into a particular, possibly esoteric, genre. If users are found to be similiar based
on their ratings, then we can be more sure that when it comes to unpopular movies, they are more likely to have
a similar opinion.
This observation based on popularity is a useful observation in a business setting. Our model is more powerful
if we use it on the half of the data that constitute "unpopular" movies.


within top 50% of users ranked by how many movies they have rated:
total number of predictions made: 8330
standard error from true rating: 0.946 out of 5


within bottom 50% of users ranked by how many movies they have rated:
total number of predictions made: 1168
standard error from true rating: 1.059 out of 5

We observe that if a user is more prolific, that is, he or she has rated more movies, the standard error for predictions
on his or her ratings is slightly lower. And if a user is less prolific, the standard error on his or her predictions
is slightly higher. This is certainly expected. This phenomenon is due to the fact that if a user has rated
more movies, the algorithm has more information about him or her preferences: the algorithm can match "nearest
neighbors" to him or her more accurately. If the nearest neighbors to a user are matched accurately, the predictions
made on the ratings will have less error.

Additional segmentation based on users:
within top 90% of users ranked by how many movies they have rated:
total number of predictions made: 4783
standard error from true rating: 0.916 out of 5

From a business perspective, this observation gives us information that users who are more prolific will obtain
better predictions. We may want to act by deploying our model on only the more prolific users. Additionally,
as a business it helps to create incentives for users to give more ratings, which can be done in a variety of ways.
Some ways include prompting the user to rate a movie rather than passively waiting for a rating, and actively
informing the user that giving more ratings leads to better recommendations. Other options can be related to
the UX design of the rating mechanism. It is best if the design is optimal for engagement.


"""

In [321]:
"""
load movie genre data
"""

original = []

with open('movies.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for i, row in enumerate(spamreader):
        original.append(row)
genres = {}
for i in range(len(original)):
    try:
        genres[original[i][0]] = original[i][5].split("|")[0]
    except:
        try:
            genres[original[i][0]] = original[i][4].split("|")[0]
        except:
            try:
                genres[original[i][0]] = original[i][3].split("|")[0]
            except:
                genres[original[i][0]] = original[i][2].split("|")[0]
#print (genres)

In [322]:
"""
put data into proper format: pivot table by doing grouby on userID columns
and append movieID row
"""
data = []

with open('ratings.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for i, row in enumerate(spamreader):
        data.append(row)

data = np.asarray(data)
data = pd.DataFrame(data=data[1:,:], columns=data[0,0:])

data = data.drop('timestamp', 1)

#print (data.head)

#print (data.head)
#print (data.columns.values)

data = data.pivot(index='userId', columns='movieId', values='rating')#['rating']

#print (data.iloc[2,:])
#print (data.columns[:])
#print (len(data.columns[:]))

cols = data.columns[:]

data = np.asarray(data)



data = np.vstack((cols,data))

print (data)

[['1' '10' '100' ..., '99912' '99917' '99992']
 [None None None ..., None None None]
 [None None None ..., None None None]
 ..., 
 ['1' None None ..., None None None]
 [None None None ..., None None None]
 ['4' None None ..., None None None]]


In [323]:
"""
append column of most watched movie genre to dataset
"""
vector = []
for i in range(1, len(data)):
    temp = []
    for j in range(len(data[i])):
        if data[i][j] != None:
            temp.append(data[0][j])
    for k in range(len(temp)):
        t = temp[k]
        a = genres[t]
        temp[k] = a
    mode = max(set(temp), key=temp.count)
    vector.append(mode)

print (len(vector))
#print (vector)

print (set(vector))

for i in range(len(vector)):
    if vector[i] == 'Horror':
        vector[i] = 10
    if vector[i] == 'Crime':
        vector[i] = 20
    if vector[i] == 'Drama':
        vector[i] = 30
    if vector[i] == 'Adventure':
        vector[i] = 40
    if vector[i] == 'Action':
        vector[i] = 50
    if vector[i] == 'Comedy':
        vector[i] = 60
#print (vector)

data = np.delete(data, 0, 0)

vector = np.asarray(vector)
vector = np.reshape(vector, (len(vector), 1))

data = np.append(data, vector, axis=1)


print (data)

671
set(['Horror', 'Crime', 'Drama', 'Adventure', 'Action', 'Comedy'])
[[None None None ..., None None 30]
 [None None None ..., None None 50]
 ['4' None None ..., None None 50]
 ..., 
 ['1' None None ..., None None 60]
 [None None None ..., None None 60]
 ['4' None None ..., None None 30]]


In [324]:
"""
create training and evaluation sets
"""

import random

evaluation_set_size = 100

initial_indices = []
for i in range(data.shape[0]):
    initial_indices.append(i)

indices = []
while len(indices) < evaluation_set_size:
    number = random.choice(initial_indices)
    if number not in indices:
        indices.append(number)

data = np.asarray(data)
print (data.shape)
eval = []
for i in range(data.shape[0]):
    if i in indices:
        eval.append(data[i][:])
eval = np.asarray(eval)
print (eval.shape)
print (eval)
data = np.delete(data, indices, axis=0)
print (data.shape)

(671L, 9067L)
(100L, 9067L)
[[None None None ..., None None 50]
 [None None None ..., None None 30]
 ['4' None None ..., None None 60]
 ..., 
 ['3' None None ..., None None 50]
 ['4' '3' None ..., None None 50]
 ['4' None None ..., None None 30]]
(571L, 9067L)


In [325]:
"""
strip off ratings from the evaluation set
"""
import copy

ratio_of_ratings_missing = 0.5

full_ratings_eval = copy.deepcopy(eval)
for j, row in enumerate(eval):
    indices = []
    for i, rating in enumerate(row):
        if rating != None and i!=9066:
            indices.append(i)
    total_ratings = len(indices)
    while len(indices) > total_ratings*ratio_of_ratings_missing:
        number = random.choice(indices)
        indices.remove(number)
    for i, rating in enumerate(row):
        if i in indices:
            eval[j][i] = None

In [326]:
"""
identify k nearest neighbors of every user in the evaluation set
"""

k = 5

dictionaries = [None] * evaluation_set_size
for i, eval_row in enumerate(eval):
    dictionaries[i] = {}
    for ref_row in data:
        measure = 0
        count = 0
        for j in range(len(eval_row)):
            if eval_row[j] != None and ref_row[j] != None:
                count += 1
                measure += (float(eval_row[j]) - float(ref_row[j]))**2
        if len(dictionaries[i]) < 5 and count>0:
            #print (measure)
            dictionaries[i][tuple(ref_row.tolist())] = measure
        else:
            if count>0 and measure < max(dictionaries[i].values()):
                #print ("measure")
                #print (dictionaries[i].values())
                #print (measure)
                #print (max(dictionaries[i].values()))
                for key in list(dictionaries[i].keys()):
                    if dictionaries[i][key] == max(dictionaries[i].values()):
                        del dictionaries[i][key]
                        break
                dictionaries[i][tuple(ref_row.tolist())] = measure
                #print (dictionaries[i].values())

In [327]:
"""
make predictions on missing ratings based on nearest neighbors
"""

record = copy.deepcopy(eval)

for i, eval_row in enumerate(eval):
    for j in range(len(eval_row)):
        if eval_row[j] == None and full_ratings_eval[i][j] != None:
            total = 0
            the_sum = 0
            for v in dictionaries[i].keys():
                if v[j] != None:
                    the_sum += float(v[j])
                    total += 1
            if total == 0:
                eval[i][j] = 3
            else:
                eval[i][j] = float(the_sum)/float(total)

In [328]:
"""
calculate standard error among predicted ratings
"""

count = 0
error = 0
for i in range(len(eval)):
    for j in range(len(eval[i])):
        if eval[i][j] != None and record[i][j] == None:
            count += 1
            error += abs(float(full_ratings_eval[i][j]) - float(eval[i][j]))
            #print ("original")
            #print (full_ratings_eval[i][j])
            #print (eval[i][j])
print (count)
print (float(error)/float(count))

7730
0.914505174644


In [ ]:
"""
In my experiment I achieved a better working model by adding side information. I used data provided by
https://grouplens.org/datasets/movielens/ which gives the genre for each movie in the dataset. There could
be many ways to incorporate this information and it is tricky to do so properly. My method was as follows:
For each user, I computed the most common movie genre that they liked to watch. Then, I added a column in the
dataset, that is, a new feature for each user, and it described each user's most watched genre. In order to work
with nearest neighbors method, these values had to be numerical, so I used a simple dictionary conversion for each
genre to a number -- and I made the numbers relatively large (e.g. 10, 20, 30) compared to the ratings. This
makes the feature stick out. To be precise, differences between users' most watched genres will mean that they
are very different users in the algorithm, and vice versa. So as you compare users based on ratings of movies watched,
you likewise compare users based on which genre they watch the most. Thus your nearest neighbors are collected.


Improvement on benchmark by adding side information on genre:
total number of predictions made: 7730
standard error from true rating: 0.914 out of 5


In summary, I have built a custom nearest neighbors algorithm from scratch. My insights on the performance relate
to the activity of users and the characteristics of movies. And they make logical, intuitive sense and can be
used to back business opinions as well as motivate business decisions. The improvement I discovered uses
side information about the movie's genre, offered by the source website. Adding a favorite genre user information
was beneficial to the model.

This concludes the model building and experimentation.
"""